<a href="https://colab.research.google.com/github/dfan14051/SUPA-HOT-GAN/blob/master/SUPAHOTGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**GAN**

In [0]:
from google.colab import files

In [0]:
uploaded = files.upload()
# Upload dataset. Download from Readme.md if needed.
# TAKES A LONG TIME

In [0]:
print(len(uploaded))

Sample GAN from: https://skymind.ai/wiki/generative-adversarial-network-gan for use on MNIST images to show a working GAN. TURN ON TPU RUNTIME!

In [0]:
import os

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

In [0]:
class GAN():
  def __init__(self):
    self.img_rows = 28
    self.img_cols = 28
    self.channels = 1
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    optimizer = Adam(0.0002, 0.5)
    
    # Discriminator
    self.discriminator = self.build_discriminator()
    self.discriminator.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])
    
    # Generator
    self.generator = self.build_generator()
    self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    z = Input(shape=(100,))
    img = self.generator(z)
    
    self.discriminator.trainable = False
    
    valid = self.discriminator(img)
    
    self.combined = Model(z, valid)
    self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
  def build_generator(self):
    noise_shape = (100,)
    
    model = Sequential()
    
    model.add(Dense(256, input_shape = noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(self.img_shape), activation='tanh'))
    model.add(Reshape(self.img_shape))
    
    model.summary()
    
    noise = Input(shape=noise_shape)
    img = model(noise)
    
    return Model(noise,img)
    
  def build_discriminator(self):
    img_shape = (self.img_rows, self.img_cols, self.channels)
    model = Sequential()
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    
    img = Input(shape=img_shape)
    validity = model(img)
    
    return Model(img, validity)
  
  def train(self, epochs, batch_size=128, save_interval=50):
    (X_train, _), (_,_) = mnist.load_data()
    
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = np.expand_dims(X_train, axis=3)
    
    half_batch = int(batch_size / 2)
    
    for epoch in range(epochs):
      # Discriminator
      idx = np.random.randint(0, X_train.shape[0], half_batch)
      imgs = X_train[idx]
      
      noise = np.random.normal(0, 1, (half_batch, 100))
      
      gen_imgs = self.generator.predict(noise)
      
      d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      
      # Generator
      noise = np.random.normal(0, 1, (batch_size, 100))
      valid_y = np.array([1] * batch_size)
      
      g_loss = self.combined.train_on_batch(noise, valid_y)
      
      # plot progress
      print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
      
      # If at save interval => save generated image samples
      if epoch % save_interval == 0:
         self.save_imgs(epoch)
          
  def save_imgs(self, epoch):
      r, c = 5, 5
      noise = np.random.normal(0, 1, (r * c, 100))
      gen_imgs = self.generator.predict(noise)

      # Rescale images 0 - 1
      gen_imgs = 0.5 * gen_imgs + 0.5

      fig, axs = plt.subplots(r, c)
      cnt = 0
      for i in range(r):
          for j in range(c):
              axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
              axs[i,j].axis('off')
              cnt += 1
      fig.savefig("mnist_%d.png" % epoch)
      files.download("mnist_%d.png" % epoch)
      plt.close()


In [68]:
gan = GAN()
gan.train(epochs=30000, batch_size=32, save_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_23 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_154 (Dense)            (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_110 (LeakyReLU)  (None, 512)               0         
_________________________________________________________________
dense_155 (Dense)            (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_111 (LeakyReLU)  (None, 256)               0         
_________________________________________________________________
dense_156 (Dense)            (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.931975, acc.: 40.62%] [G loss: 0.694058]
1 [D loss: 0.522342, acc.: 65.62%] [G loss: 0.755265]
2 [D loss: 0.318496, acc.: 90.62%] [G loss: 0.793288]
3 [D loss: 0.297891, acc.: 81.25%] [G loss: 0.909745]
4 [D loss: 0.270558, acc.: 96.88%] [G loss: 1.019317]
5 [D loss: 0.222558, acc.: 96.88%] [G loss: 1.114606]
6 [D loss: 0.197650, acc.: 96.88%] [G loss: 1.207350]
7 [D loss: 0.185886, acc.: 100.00%] [G loss: 1.311875]
8 [D loss: 0.203679, acc.: 93.75%] [G loss: 1.438459]
9 [D loss: 0.151404, acc.: 100.00%] [G loss: 1.486858]
10 [D loss: 0.164593, acc.: 96.88%] [G loss: 1.626907]
11 [D loss: 0.137754, acc.: 100.00%] [G loss: 1.690046]
12 [D loss: 0.108787, acc.: 100.00%] [G loss: 1.838070]
13 [D loss: 0.123447, acc.: 100.00%] [G loss: 1.856068]
14 [D loss: 0.107272, acc.: 100.00%] [G loss: 1.874792]
15 [D loss: 0.114519, acc.: 100.00%] [G loss: 1.919805]
16 [D loss: 0.068169, acc.: 100.00%] [G loss: 2.010343]
17 [D loss: 0.092346, acc.: 100.00%] [G loss: 2.101726]
18 [D loss:

KeyboardInterrupt: ignored

Modified GAN for CIFAR10

In [0]:
from keras.datasets import cifar10

In [0]:
class GAN():
  def __init__(self):
    self.img_rows = 32
    self.img_cols = 32
    self.channels = 3
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    optimizer = Adam(0.0002, 0.5)
    
    # Discriminator
    self.discriminator = self.build_discriminator()
    self.discriminator.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])
    
    # Generator
    self.generator = self.build_generator()
    self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    z = Input(shape=(900,))
    img = self.generator(z)
    
    self.discriminator.trainable = False
    
    valid = self.discriminator(img)
    
    self.combined = Model(z, valid)
    self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
  def build_generator(self):
    noise_shape = (900,)
    
    model = Sequential()
    model.add(Dense(256, input_shape = noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(2048))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(self.img_shape), activation='tanh'))
    model.add(Reshape(self.img_shape))
    
    model.summary()
    
    noise = Input(shape=noise_shape)
    img = model(noise)
    
    return Model(noise,img)
    
  def build_discriminator(self):
    img_shape = (self.img_rows, self.img_cols, self.channels)
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=img_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    
    img = Input(shape=img_shape)
    validity = model(img)
    
    return Model(img, validity)
  
  def train(self, epochs, batch_size=128, save_interval=50):
    (X_train, _), (_,_) = cifar10.load_data()
    
    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    #X_train = np.expand_dims(X_train, axis=3)
    
    half_batch = int(batch_size / 2)
    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for epoch in range(epochs):
      # Discriminator
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]
      
      noise = np.random.normal(0, 1, (batch_size, 900))
      
      gen_imgs = self.generator.predict(noise)
      
      d_loss_real = self.discriminator.train_on_batch(imgs, valid)
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      
      # Generator
      noise = np.random.normal(0, 1, (batch_size, 900))
      valid_y = np.array([1] * batch_size)
      
      g_loss = self.combined.train_on_batch(noise, valid_y)
      
      # plot progress
      #print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
      
      # If at save interval => save generated image samples
      if epoch % save_interval == 0:
         self.save_imgs(epoch)
          
  def save_imgs(self, epoch):
      r, c = 5, 5
      noise = np.random.normal(0, 1, (r * c, 900))
      gen_imgs = self.generator.predict(noise)

      # Rescale images 0 - 1
      gen_imgs = 0.5 * gen_imgs + 0.5

      fig, axs = plt.subplots(r, c)
      cnt = 0
      for i in range(r):
          for j in range(c):
              axs[i,j].imshow(gen_imgs[cnt])
              axs[i,j].axis('off')
              cnt += 1
      fig.savefig("cifar10_%d.png" % epoch)
      files.download("cifar10_%d.png" % epoch)
      plt.close()

In [81]:
gan = GAN()
gan.train(epochs=30000, batch_size=32, save_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_19 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_20 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 15, 15, 64)        18496     
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


**INSTALL MAGENTA**

In [0]:
#@test {"output": "ignore"}

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

# Temporary hack since the colab installs a RC version of tensorflow.
!pip uninstall -y tensorflow
!pip install -q magenta-gpu

# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta.music as mm
import magenta
import tensorflow

print('🎉 Done!')
print(magenta.__version__ )
print(tensorflow.__version__)